In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

22/12/20 14:05:41 WARN Utils: Your hostname, notebook resolves to a loopback address: 127.0.1.1; using 10.0.0.67 instead (on interface enp0s3)
22/12/20 14:05:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 14:05:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('course-material/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [15]:
for i in data.head(2)[1]:
    print(i)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [16]:
from pyspark.ml.linalg import Vector

In [17]:
from pyspark.ml.feature import VectorAssembler

In [19]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [22]:
assembler = VectorAssembler(inputCols=[
    'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol='features')

In [23]:
output = assembler.transform(data)

In [25]:
output.select(output['features']).show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [26]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [27]:
final_data = output.select('features', 'Yearly Amount Spent')

In [28]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [29]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [30]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                375|
|   mean|   499.555537282239|
| stddev|  80.89321817083423|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [33]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                125|
|   mean|  498.5895411876466|
| stddev|  74.68700290060804|
|    min|  304.1355915788555|
|    max|  708.9351848669818|
+-------+-------------------+



In [34]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [35]:
lr_model = lr.fit(train_data)

22/12/20 14:17:49 WARN Instrumentation: [184c6c4f] regParam is zero, which might cause numerical instability and overfitting.
22/12/20 14:17:49 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [36]:
test_results = lr_model.evaluate(test_data)

In [37]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -3.108522167208946|
|-0.26701997294787816|
|   23.47101246305988|
|  19.540621695511675|
|  3.5444951004096197|
|  -4.835391304199902|
|  -8.599602360670076|
|  3.9717239328672918|
| -13.266722765835311|
|  18.782411883772284|
|  -1.589313666626765|
|  -8.889326343672053|
| -1.1220989288079863|
|   5.277084000525747|
|  0.6832999038121557|
|    5.46606456806046|
|   1.383205381296193|
|  23.044982754695468|
|   17.79777810027292|
| -1.7170522597698437|
+--------------------+
only showing top 20 rows



In [38]:
test_results.rootMeanSquaredError

9.398328874058477

In [39]:
test_results.r2

0.9840375503649084

In [40]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [41]:
unlabeled_data = test_data.select('features')

In [42]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8364326747734...|
|[31.2606468698795...|
|[31.2834474760581...|
|[31.3123495994443...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.7216523605090...|
|[31.8279790554652...|
|[31.8627411090001...|
|[31.9764800614612...|
|[32.0047530203648...|
|[32.0215955013870...|
|[32.0444861274404...|
|[32.0498393904573...|
|[32.0609143984100...|
|[32.0775900443291...|
+--------------------+
only showing top 20 rows



In [44]:
predictions = lr_model.transform(unlabeled_data)

In [46]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...|470.61042259419855|
|[31.2606468698795...|421.59365122989925|
|[31.2834474760581...| 568.3100769626076|
|[31.3123495994443...|444.05079633242894|
|[31.4459724827577...|481.33246983471895|
|[31.5147378578019...| 494.6478793006613|
|[31.5261978982398...| 417.6941285530079|
|[31.5316044825729...|432.54388179649527|
|[31.5741380228732...| 557.6759949264222|
|[31.6005122003032...|460.39043960732465|
|[31.7216523605090...| 349.3662402984994|
|[31.8279790554652...| 448.8920738906136|
|[31.8627411090001...| 557.4202401028547|
|[31.9764800614612...|325.31736203357445|
|[32.0047530203648...| 463.0626812168173|
|[32.0215955013870...| 516.1061101897669|
|[32.0444861274404...| 446.8466238052538|
|[32.0498393904573...| 455.6743741195198|
|[32.0609143984100...| 609.8055406127421|
|[32.0775900443291...| 402.7501874788759|
+--------------------+------------